# <center>Llama 3.2 介绍与部署流程

前言

LLaMA 3 系列模型是由 Meta AI 研发的第三代大规模语言模型，旨在延续其前代模型 LLaMA 和 LLaMA 2 的优良性能，并在多个方面进行进一步提升。LLaMA 3 在设计上采用了更深层的架构和更广泛的数据集，以便更好地处理复杂的自然语言处理任务。

LLaMA 3 已经被广泛应用于各种自然语言处理应用场景，包括文本生成、机器翻译、情感分析等。其开源性也为研究人员和开发者提供了便利，使他们能够根据自身需求对模型进行进一步的微调和优化。性能测试表明，LLaMA 3 系列在多个基准测试上均取得了优异成绩，尤其在生成式任务上，其表现不亚于其他同类领先的大型语言模型，具有更快的推理速度和更高的输出质量。

目录

一 Llama 3.2 介绍与部署流程<br>
&emsp;1 Llama 3.2系列版本介绍<br>
&emsp;2 线上Llama 3.2：3B测试<br>
&emsp;3 线上Llama 3.2:11B version版本测试<br>
二 Llama 3.2部署流程<br>
&emsp;2.1 官网流程下载<br>
&emsp;2.1.1 硬件配置<br>
&emsp;2.1.2 虚拟环境搭建<br>
&emsp;2.1.3 下载工具初始化<br>
&emsp;2.1.4 下载模型文件<br>
&emsp;2.2 Ollama方式下载<br>
&emsp;2.2.1 Ollama方式实现Llama3.2下载全流程<br>
&emsp;2.2.2 如何管理&卸载Ollama相关模型<br>
&emsp;2.3 在Windows环境下进行Ollama部署与下载模型<br>

## 1. Llama 3.2模型介绍

### 1.1具体版本介绍

就在Llama 3.1发布的不到2个月，Meta官方便推出了Llama 3.2系列模型，以下是它们的主要版本描述：

Llama 3.2 Vision 提供两种尺寸的模型：11B，适合在消费级 GPU 上高效部署和开发；90B，适合大规模应用。这两种版本都包括基础版和指令微调版。除了这四个多模态模型外，Meta 还发布了新版支持视觉的 Llama Guard。Llama Guard 3 是一款安全模型，能够对模型输入和生成内容进行分类，包含检测有害的多模态提示或助手响应的功能。

此外，Llama 3.2 还包括可在设备上运行的小型仅文本语言模型，分为 1B 和 3B 两种新尺寸，提供基础版和指令版，具备强大的功能。此外，还有一个小型 1B 版本的 Llama Guard，可与这些文本模型或更大的模型结合用于生产场景。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241018162839343.png" width=60%></div>

- Llama 3.2 vision

Llama 3.2 Vision 是 Meta 发布的最强大的开源多模态模型，具备出色的视觉理解和推理能力，能够处理各种任务，例如视觉推理与定位、文档问答，以及图像-文本检索。该模型在回答思维链 (Chain of Thought, CoT) 问题时表现优异，使其在视觉推理方面尤为强大。

Llama 3.2 Vision 可以同时处理文本和图像，也可以仅处理文本。对于图像-文本提示，模型接受英文输入；而在仅文本提示时，支持包括英语、德语、法语、意大利语、葡萄牙语、印地语、西班牙语和泰语在内的多种语言。

该模型的架构是基于 Llama 3.1 LLM，结合了视觉塔和图像适配器。Llama 3.2 11B Vision 使用的是 Llama 3.1 8B 版本的文本模型，而 Llama 3.2 90B Vision 则搭载了 Llama 3.1 70B 版本的文本模型。文本模型在训练视觉模型期间是冻结的，以确保其在仅文本模式下的性能保持不变。


Vision 模型的上下文长度为 128k 个 token，使其能够处理包含图像的多轮对话。不过，该模型在处理单一图像时表现最佳，因此在 transformers 实现中，仅关注输入中的最后一张图像，以便保持结果质量并节省内存。

11B 的基础模型支持分块尺寸为 448，而指令微调版本和 90B 模型则使用 560 的分块尺寸。这些模型在一个包含 60 亿图文对的海量数据集上进行了训练，数据来源广泛多样。这种训练方式使得这些模型非常适合用于下游任务微调。Meta 提供了一些基准测试结果，以展示 11B 和 90B 模型及其指令微调版本的表现。

- 系统（system）：设置与 AI 模型交互的上下文。它通常包括帮助模型有效响应的规则、准则或必要信息。

- 用户（user）：表示与模型交互的人类。它包括模型的输入、命令和问题。

- 编程（ipython）：Llama 3.1 中引入的新角色。从语义上讲，这个角色的意思是 “工具”。此角色用于在从 executor 发送回模型时，使用 tool call 的输出标记消息。

- 助手(assistant)：表示 AI 模型根据系统、ipython 和用户提示中提供的上下文生成的响应。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017184443502.png" width=80%></div>

- MMMU (Micro Average Accuracy)：

Micro Average Accuracy：用于评估模型在多模态理解任务中的准确性。该任务可能包括文本和图像的联合理解。
这里列出了基于不同条件（如 CoT 思维链、零样本推理）的准确率，表示模型在处理多轮对话和推理任务时的表现。

- VQAv2 (Accuracy)：

Accuracy：在视觉问答（Visual Question Answering, VQA）任务中的准确率。VQAv2 是一种常见的视觉问答数据集，用于评估模型根据图像回答问题的能力。
该数据集有两个部分：验证集（val）和测试集（test）。数值表示模型在这两部分数据集上的准确性。

- DocVQA (ANLS)：

ANLS (Average Normalized Levenshtein Similarity)：用于文档视觉问答（Document Visual Question Answering, DocVQA）任务，专注于文档中的文本提取与问答。
ANLS 是一种特定的度量方法，衡量模型的回答与正确答案之间的相似性，数值越高表示模型的回答越接近真实答案。
这里列出了模型在验证集（val）和测试集（test）上的 ANLS 分数，表明其在文本识别和问答任务中的表现。

- AI2D：

AI2D：可能指的是 Allen Institute for AI Development 的一些特定任务或数据集，如与图形理解或多模态数据相关的任务。具体的指标和评估方式要根据该模型在 AI2D 中的任务定义而定。

>值得注意的是，3B 模型在 IFEval 上的表现几乎与 8B 模型相当，这使其非常适合代理类应用场景，在这些场景中，严格遵循指令对于可靠性至关重要。对于这个规模的模型来说，获得如此高的 IFEval 分数尤为令人瞩目。

>1B 和 3B 的指令调优版本都支持工具的使用。用户可以在 0-shot 环境中直接指定使用工具，即使模型并没有提前关于开发者将使用工具的具体信息。因此，Llama 3.1 模型内置的工具（如 brave_search 和 wolfram_alpha）在这里不再可用。

- Llama  3.2 1B & 3B

Llama 3.2 系列包含 1B 和 3B 的文本模型，专为设备上的使用场景设计，例如提示重写、多语言知识检索、摘要、工具使用以及本地运行的助手。这些模型在小规模下性能卓越，超越了许多开放访问模型，甚至能够与更大规模的模型相竞争。

这些模型沿用了 Llama 3.1 的架构，训练数据量达到 9 万亿个 token，支持长达 128k token 的上下文长度。它们是多语言的，支持包括英语、德语、法语、意大利语、葡萄牙语、印地语、西班牙语和泰语在内的多种语言。

此外，Llama Guard 的新小版本——Llama Guard 3 1B，可与这些模型一同部署，以评估多轮对话中用户或助手的最新响应。它具备一组预定义的类别（此版本新增），可以根据开发者需求进行自定义或排除。该大小的版本适合应用于文本摘要，语言翻译等任务。



>提示：Llama 3.2 涉及的语言范围超出上述 8 种，鼓励开发者根据特定语言的需求对 Llama 3.2 进行微调。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241009145248359.png" width=80%></div>

据官方所说运行Llama 3.2：1B模型，推荐使用 `PyTorch ExecuTorch `框架。`ExecuTorch `是一个端到端的解决方案，旨在在移动设备和边缘设备（如可穿戴设备、嵌入式设备和微控制器）上实现推理功能。作为 PyTorch Edge 生态系统的一部分，它支持高效地将各种` PyTorch `模型（包括视觉、语音和生成式 AI 模型）部署到边缘设备上。

更多信息可以访问以下网址：
https://hf.co/blog/llama32

### 1.2 线上Llama 3.2：3B测试

线上测试：
https://huggingface.co/spaces/huggingface-projects/llama-3.2-3B-Instruct

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241015174942369.png" width=80%></div>

在下面对话框输入文本即可实现对话，可见Llama 3.2：3B模型具有多语言和记忆能力。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017180318748.png" width=80%></div>

### 1.2 线上Llama 3.2：11B测试

线上测试：https://huggingface.co/spaces/huggingface-projects/llama-3.2-vision-11B

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017190545912.png" width=80%></div>

llama-3.2-vision-11B模型支持输入图片和文字信息，通过分析输入信息返回文本内容。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241018154230749.png" width=80%></div>

## 2. Llama 3.2部署流程

### 2.1 官网下载部署流程

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404171645681.png" alt="开源大模型下载方法" style="zoom:100%;" />

&emsp;&emsp;大模型的部署流程和方法大致相似，主要包括创建Python虚拟环境、下载项目文件、下载模型权重以及具体使用这四个关键步骤。同时，在私有化部署大模型的过程中，尽管CPU可以加载并运行大模型但其效率非常低。因此建议使用GPU来加载模型以提高效率。

### 2.1.1 硬件配置

&emsp;&emsp;在进行实际部署之前，大家需要根据上述的显存占用情况，合理选择适合部署的Llama 3.2:3B模型，避免由于硬件限制导致模型无法成功加载。

&emsp;&emsp;**本文实验教程采用的机器配置为 4 张 NVIDIA GeForce RTX 3090，选择的模型为Llama 3.2:3B模型在Ubuntu 22.04版本下进行安装部署，接下来，我们逐步介绍私有化部署的全部流程。**

> 注：如果部署环境是租赁的云服务器，显卡驱动环境已经预设好，并不需要我们再次手动配置。

&emsp;&emsp;如果Linux系统配备了至少16GB VRAM的GPU，应该能够在本地加载fp16格式的8B Llama模型。可以使用NVIDIA系统管理界面工具确认设置，该工具会显示系统拥有的GPU、可用的VRAM以及其他有用信息。通过键入以下命令可以查看这些详细信息：

nvidia-smi

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011103224147.png" width=60%></div>

### 2.1.2 虚拟环境搭建

- **Step 1. 创建conda虚拟环境**

&emsp;&emsp;Conda创建虚拟环境的意义在于提供了一个隔离的、独立的环境，用于Python项目和其依赖包的管理。每个虚拟环境都有自己的Python运行时和一组库。这意味着我们可以在不同的环境中安装不同版本的库而互不影响。根据官方文档信息建议Python版本3.10以上。创建虚拟环境的办法可以通过使用以下命令创建：

```bash
# 其中llama_3_1 是你想要给环境的名称，python=3.11 指定了要安装的Python版本。你可以根据需要选择不同的名称和/或Python版本。

conda create -n llama_3_1 python==3.11
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011111418418.png" width=80%></div>

&emsp;&emsp;创建虚拟环境后，需要激活它。使用以下命令来激活刚刚创建的环境。如果成功激活，可以看到在命令行的最前方的括号中，就标识了当前的虚拟环境。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011111501146.png" width=80%></div>

&emsp;&emsp;如果忘记或者想要管理自己建立的虚拟环境，可以通过conda env list命令来查看所有已创建的环境名称。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240928143512354.png" width=80%></div>

&emsp;&emsp;如果需要卸载指定的虚拟环境则通过以下指令实现：
```
conda env remove --name envname
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240928143643575.png" width=80%></div>

- 需要注意的是无法卸载当前激活的环境，建议卸载时先切换到base环境中再执行操作。

- **Step 2. 查看当前驱动最高支持的CUDA版本**

&emsp;&emsp;我们需要根据CUDA版本选择Pytorch框架，先看下当前的CUDA版本：
```
nvidia -smi
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924161818464.png" width=70%></div>

- **Step 3. 在虚拟环境中安装Pytorch**

&emsp;&emsp;进入Pytorch官网：https://pytorch.org/get-started/previous-versions/

大家需要根据自己的CUDA版本情况下载适合的Pytorch版本，选择版本接近且不大于主机CUDA所支持的最高版本即可。

<div align=center><img src="https://muyu001.oss-cn-beijing.aliyuncs.com/img/image-20240103163206436.png" width=80%></div>

&emsp;&emsp;当前的电脑CUDA的最高版本要求是12.2，所以需要找到不大于12.2版本的Pytorch。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401041455184.png" width=80%></div>

&emsp;&emsp;直接复制对应的命令，进入终端执行即可。这实际上安装的是为 CUDA 12.1 优化的 PyTorch 版本。这个 PyTorch 版本预编译并打包了与 CUDA 12.1 版本相对应的二进制文件和库。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011112225733.png" width=80%></div>

- **Step 4. 安装Pytorch验证**

&emsp;&emsp;待安装完成后，如果想要检查是否成功安装了GPU版本的PyTorch，可以通过几个简单的步骤在Python环境中进行验证：
```bash
import torch

print(torch.__version__)
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011112443325.png" width=80%></div>

&emsp;&emsp;如果输出是版本号数字，则表示GPU版本的PyTorch已经安装成功并且可以使用CUDA，如果显示ModelNotFoundError，则表明没有安装GPU版本的PyTorch，或者CUDA环境没有正确配置，此时根据教程，重新检查自己的执行过程。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241012105117779.png" width=80%></div>

&emsp;&emsp;当然通过pip show的方式可以很简洁的查看已安装包的详细信息。pip show <package_name> 可以展示出对应安装包的版本、依赖关系（展示一个包依赖哪些其他包）、定位包安装位置、验证安装确实包是否正确安装及详情。

### 2.1.3 下载工具初始化

在正式下载工具之前需要对下载工具进行检查和升级，如果使用的是旧版本的安装工具，可能无法安装一些最新的包，或者可能无法正确解析依赖关系。

- **Step 5. 升级pip版本**

升级 pip 很简单，只需要运行命令如下命令：

python -m pip install --upgrade pip 

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241012110959943.png" width=80%></div>

- **Step 6. 检查和更新wget和md5sum**

同样的还需要对以下两个安装工具进行检查和更新wget和md5sum，首先可以先检查以下你的系统是否已经预安装了这两个工具：

```
wget --version
md5sum --version
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011112646376.png" width=60%></div>

如果没有该工具，可以通过以下指令进行安装：
```
apt-get install wget
apt-get install md5sum
```

### 2.1.4 下载模型文件

- **Step 7. 创建工作文件夹**

在终端的工作区中创建一个名为`llama3_1`的新文件夹。导航到新文件夹并克隆 Llama 存储库

```
mkdir llama3_2
cd llama3_2
```

- **Step 8. 在官网获取下载权重模型权限**

要下载权重，可访问Llama网站。在表格中填写详细信息并选择需要下载的模型。本例中将下载Llama 3模型。值得注意的是，虽然表格中的信息可以自行拟定，但国家/地区选项需要与VPN所在地对应，否则无法进行后续的授权步骤。

官网下载连接：https://www.llama.com/llama-downloads/

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017162029780.png" width=100%></div>

完成注册之后点击当页面的next按钮进入协议页面，阅读并同意许可协议后，点击接受并继续。随后网站上将显示一个唯一的URL。该URL也会通过电子邮件发送，**有效期为24小时，允许每个模型最多下载5次**。如需要，可随时请求新的URL。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011105812338.png" width=100%></div>

同时你所填写的申请邮箱也会收到一封来自Meta的邮件，里面同样包含了下载时可以使用的URL信息，以及所开放的可下载模型类型，有效期也是24h。

通过以下指令下载Llama的安装插件
```
pip install llama-stack
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241012113407646.png" width=100%></div>

同过指令可以查看所有支持的模型版本信息。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241012113440992.png" width=60%></div>

通过以下指令可以开启模型的下载，需要在对应的模型信息中填写要下载的具体模型名称，随后会要求输入授权的URL信息，将邮件或授权成功界面的信息复制在下面即可。

llama model download --source meta --model-id  MODEL_ID

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017162516124.png" width=80%></div>

如果遇到以下报错，再次返回下载授权页面重新选择授权即可。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241012115552782.png" width=80%></div>

- **Step 9. 模型启动**

现已为运行示例推理脚本做好准备，以测试模型是否已正确设置并能正常工作。Meta官方创建了一个名为` example_text_completion.py` 的示例 Python 脚本可用于测试模型。注意需要将 ckpt_dir 后面的信息替换为检查点目录的路径，将 tokenizer_path 后面的信息替换为 tokenizer 模型的路径。如果从此主目录运行它则可能不需要更改路径。

输入以下指令启动测试脚本：
```
torchrun --nproc_per_node 1 example_text_completion.py --ckpt_dir Meta-Llama-3.2-1B/ --tokenizer_path Meta-Llama-3.2-1B/tokenizer.model --max_seq_len 128 --max_batch_size 4
```

` example_text_completion.py`该脚本定义了一个 main 函数，该函数使用 llama 库中的 Llama 类，通过预先训练的模型为给定的提示生成文本补全。它有以下几个超参数控制模型输出。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011113634346.png" width=80%></div>

### 2.2 Ollama方式下载

Ollama 是一个开源的大语言模型服务工具，专注于简化本地模型的创建、管理和部署流程。它可以帮助开发人员和数据科学家轻松地在本地或私有环境中使用大语言模型，而不必依赖于云服务，从而保证了数据隐私和灵活性。

Ollama 非常适合需要在本地运行大语言模型的开发者和企业，如：

- 开发和测试：在本地快速创建和测试新的语言模型。
- 隐私保护：在本地部署模型，适用于有严格隐私需求的企业。
- 多模型管理：轻松管理和部署多个模型，适合有大量模型管理需求的团队。

在官网可以看到Ollama支持的模型列表 https://ollama.com/library

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017165417465.png" width=80%></div>

每个模型下面有支持的功能和参数型号以及基本的模型描述，点击进入对应模型可以看到下载所需占用的内存大小。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017165345618.png" width=80%></div>

除了LLama系列模型，Ollama 支持下载 Gemma、qwen 和 glm4 等多种主流大语言模型和代码语言模型，下图展示了Ollama所支持的部分系列模型的具体型号以及所需要使用的命令。

### 2.2.1 使用Ollama实现LLM下载流程

使用Ollama方法下载的前两步骤与正常方法下载相同（本文2.1.1和2.1.2步骤），即创建虚拟环境和依据硬件配置Pytorch环境，部署好流程后可以继续下面的步骤。

Ollama安装硬件要求：

- Windows：3060以上显卡+8G以上显存+16G内存，硬盘空间至少20G显卡
- 
Mac：M1或M2芯片 16G内存，20G以上硬盘空间

下载Ollama的指令如下：

```
curl -fsSL https://ollama.com/install.sh | sh

```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011114550703.png" width=80%></div>

下载完成后检测,如果返回版本号则说明成功下载：
```
ollama -v
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011114741149.png" width=80%></div>

通过指令**ollama help**可以查看该系统可执行的命令：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011144419942.png" width=80%></div>

通过以下指令可以，检查ollama可运行的模型列表，可以看见之前下载的Llama 3-8b模型的信息呈现在列：

```
ollama list
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241011115126172.png" width=80%></div>

**下载模型**

在终端中执行命令 `ollama run llama3.2:3B`或`ollama run llama3.2` ，即可下载 Llama3.2:3B 模型。模型下载完成后，会自动启动大模型，进入命令行交互模式，直接输入指令，就可以和模型进行对话了对应参数的模型的下载方式可以通过在Ollama官网查看到下载指令。

以下以参数为3B的模型为例作为演示，部署该模型仅需要2G的内存。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017165345618.png" width=80%></div>

下载70B的模型只需要一行指令即可完成部署：

```
ollama run llama3.2:3b
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241015150023157.png" width=80%></div>

下载成功会有以下标识出现，时间随网速而定，需要耐心等待。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241015150150709.png" width=80%></div>

完成下载后会直接进入模型启动状态，如果退出或刷新界面，再次输入指令`ollama run llama3.2:3b`即可启动对应模型。
由于是轻量化的小模型所以推理速度非常的快。下例是一个代码能力的测试:

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241015150052781.png" width=80%></div>

再次输入`ollama list`指令可以看到所有使用Ollama下载完成的模型：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017171005914.png" width=80%></div>

>使用 /bye 退出 Ollama。

### 2.2.2 如何卸载安装

直接在你的安装目录下，删除ollama文件夹即可。所有下载的数据和大模型文件都在里面，Ollama 的默认安装目录通常是用户主目录下的 .ollama 文件夹。例如，在 Unix 系统（如 macOS 和 Linux）上，默认安装目录为：

~/.ollama


或可以通过指令 `ollama rm modlename`的方式来移除对应的大模型文件。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241017171236539.png" width=80%></div>

如图所示，通过`ollama rm llama3.2:3b`命令可以卸载3B参数模型，再用`ollama list`指令进行查看时模型列表中已没有该模型。

### 2.3 在Windows环境下进行Ollama方式下载

Ollama同样支持在Windows环境下进行部署下载，使用此方法下载意味着你可以无需调整任何硬件设备直接在你的主机上实现Llama 3.2模型的安装部署和推理使用。

首先要下载Windows版的Ollama文件，进入官网选择下图的内容进行下载：

https://ollama.com/download

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241015180846073.png" width=80%></div>

点击Download for window即可实现下载安装包，文件大小为664MB。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241015181037931.png" width=60%></div>

下载完成后在对应文件夹双击打开安装包完成安装流程。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241015184929499.png" width=80%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016102814619.png" width=80%></div>

框架和模型默认路径都是C盘，为了主机内存管理方便，可以通过以下步骤来实现模型下载地址的切换。


1.在桌面下方搜索栏输入高级系统设置并进入。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016103219333.png" width=80%></div>

在系统设置中找到`高级`的标签，点击下方的环境变量选项。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016103304360.png" width=80%></div>

在系统变量栏点击新建，变量名设置为OLLAMA_MODELS,变量值设置为你想要的路径位置，这里设置为D:\Ollama\models

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016103947874.png" width=80%></div>

设置好路径之后可以在命令行中进行检查，通过以下命令实现：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016104050213.png" width=80%></div>

在完成Ollama部署之后，系统会自动启动ollama的程序，界面如下：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016104224705.png" width=80%></div>

和在Linux系统中进行的指令一样，执行`ollama run llama3.2:1b`即可实现该模型的下载和运行。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016104428349.png" width=80%></div>

同样的，在完成模型下载之后会直接开启模型的推理任务，可以通过命令行的方式进行模型对话交互。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016105213302.png" width=80%></div>

再次启动Ollama可以直接在Windows的命令行中实现，通过win+R键启动cmd命令，执行`ollama run llama3.2:1b`便可以开启对话，如下图所示，虽然1B的小模型比较便捷且不消耗过多的计算资源，但是相应的推理功能并不理想。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241016112705199.png" width=80%></div>